In [1]:
# We are going to implement Eqaul Weight S&P 500 stocks
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
# Used to get the details about the stocks in our dataset
from secret import IEX_CLOUD_API_TOKEN

In [4]:
# Tryign to fetch data for one stock
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

dict

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price, market_cap / 1000000000000)

149.41 2.411248349479


In [6]:
# These are the columns that will be in our final excel sheet
chosen_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = chosen_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
# This code is SLOW as it makes an api call for every stock
# sample_df = pd.DataFrame(columns = chosen_columns)
# for stock in stocks['Ticker']:
#    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
#    data = requests.get(api_url).json()
#    price = data['latestPrice']
#    market_cap = data['marketCap']
#    temp_df = pd.DataFrame({'Ticker': [stock], 'Stock Price': [price], 'Market Capitalization': [market_cap], 'Number of Shares to Buy': ['N/A'] })
#    final_dataframe = pd.concat([final_dataframe, temp_df], ignore_index = True)


"\narr = []\nsample_df = pd.DataFrame(columns = chosen_columns)\nfor stock in stocks['Ticker']:\n    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'\n    data = requests.get(api_url).json()\n    price = data['latestPrice']\n    market_cap = data['marketCap']\n    temp_df = pd.DataFrame({'Ticker': [stock], 'Stock Price': [price], 'Market Capitalization': [market_cap], 'Number of Shares to Buy': ['N/A'] })\n    final_dataframe = pd.concat([final_dataframe, temp_df], ignore_index = True)\nfinal_dataframe\n"

In [8]:
# Function to return set of n values from a list
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i: i + n]

In [9]:
# A 2D array with each 1D array storing 100 stocks
stock_groups = list(chunks(stocks['Ticker'], 100))
# Converting these groups into comma separated string to usse it in our new api call
stock_strings = []
for i in range(len(stock_groups)):
    stock_strings.append(",".join(stock_groups[i]))
final_df = pd.DataFrame(columns= chosen_columns)
# Using batch api to make 1 call for 100 stocks which is 100x faster than the previous method
for stock_string in stock_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for stock in stock_string.split(','):
        # if stock not in received data
        if stock not in data:
            continue
        # making temporary data frame to store the current stock
        temp_df = pd.DataFrame({'Ticker': [stock], 'Stock Price': [data[stock]['quote']['latestPrice']], 'Market Capitalization': [data[stock]['quote']['marketCap']], 'Number of Shares to Buy': ['N/A']})
        # concatenating and storing it in our main data frame
        final_df = pd.concat([final_df, temp_df], ignore_index = True)
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,155.520,46977633889,N/A
1,AAL,13.840,9174153030,N/A
2,AAP,154.510,8940921881,N/A
3,AAPL,150.824,2430946743873,N/A
4,ABBV,164.730,291336514725,N/A
...,...,...,...,...
496,YUM,129.990,37554238828,N/A
497,ZBH,125.382,26306019856,N/A
498,ZBRA,280.500,14442002248,N/A
499,ZION,53.150,7996810327,N/A


In [10]:
# amount of money the trader has
portfolio_size = input('Enter the total value of your portfolio: ')
val = 0
# checking for value error
try:
    val = float(portfolio_size)
except ValueError:
    print("That is not a number \n")
    portfolio_size = input('Enter the total value of your portfolio: ')
    val = float(portfolio_size)

Enter the total value of your portfolio: 10000000


In [11]:
# Calculating the number of shares to buy for each stock based on stock price
position_size = val / len(final_df.index)
for i in range(len(final_df.index)):
     final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Stock Price'])
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,155.520,46977633889,128
1,AAL,13.840,9174153030,1442
2,AAP,154.510,8940921881,129
3,AAPL,150.824,2430946743873,132
4,ABBV,164.730,291336514725,121
...,...,...,...,...
496,YUM,129.990,37554238828,153
497,ZBH,125.382,26306019856,159
498,ZBRA,280.500,14442002248,71
499,ZION,53.150,7996810327,375


In [12]:
# Excel Writer Object is created
writer = pd.ExcelWriter("Recommended Trades.xlsx")
final_df.to_excel(writer, "Recommended Trades", engine = "openpyxl")

In [13]:
# Formatting for each column

bg_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : bg_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : bg_color,
        'border' : 1,
        'num_format' : '$0.00' 
    }
)

integer_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : bg_color,
        'border' : 1,
        'num_format' : '0' 
        
    }
)

In [14]:
# Code to format each column values manually

# writer.sheets["Recommended Trades"].set_column('B:B', 18, string_format)
# writer.sheets["Recommended Trades"].set_column('C:C', 18, string_format)
# writer.sheets["Recommended Trades"].set_column('D:D', 18, string_format)
# writer.sheets["Recommended Trades"].set_column('E:E', 18, string_format)
# writer.save()

# Code to format each column headers manually

# writer.sheets["Recommended Trades"].write('B1', 'Ticker', string_format)
# writer.sheets["Recommended Trades"].write('C1', 'Stock Price', dollar_format)
# writer.sheets["Recommended Trades"].write('D1', 'Market Capitalization', dollar_format)
# writer.sheets["Recommended Trades"].write('E1', 'Number of Shares to Buy', integer_format)



In [15]:
# Simplifying the formatting using a dictionary and for loop
columns_format = {
    'B':['Ticker', string_format],
    'C':['Stock Price', dollar_format],
    'D':['Market Capitalization', dollar_format],
    'E':['Number of Shares to Buy', integer_format]
}
for column in columns_format.keys():
    writer.sheets["Recommended Trades"].set_column(f'{column}:{column}', 18, columns_format[column][1])
    writer.sheets["Recommended Trades"].write(f'{column}1', columns_format[column][0], columns_format[column][1])

# Saving the excel writer object    
writer.save()
    

C:\Users\deepa\AppData\Local\Temp\ipykernel_35016\61595379.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
